In [1]:
# Import des bibliothèques nécessaires
from sklearn import linear_model
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# Lecture du fichier "admissions.csv"
df = pd.read_csv('admission.csv')

# Affichage des premières lignes
df.head()


,admit,gre,gpa,rank
0,0.0,380.0,3.61,3.0
1,1.0,660.0,3.67,3.0
2,1.0,800.0,4.00,1.0
3,1.0,640.0,3.19,4.0
4,0.0,520.0,2.93,4.0


In [2]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   admit   400 non-null    float64
 1   gre     400 non-null    float64
 2   gpa     400 non-null    float64
 3   rank    400 non-null    float64
dtypes: float64(4)
memory usage: 12.6 KB


In [3]:
# Suppression des instances contenant au moins une valeur nulle.
df = df.dropna()

In [4]:
df[['gre', 'admit']].groupby('admit').mean()

,gre
admit,
0.0,573.186813
1.0,618.897638


In [5]:
test_gre = pd.cut(x = df['gre'],
                  bins = [200, 450, 550, 620, 800],
                  labels = ['mauvais', 'moyen', 'moyen +', 'bon'])

pd.crosstab(df['admit'], test_gre)

# Plus le résultat obtenu au GRE est élevé, plus le candidat **semble** avoir de chances d'être admis.
# En effet, les classes d'admis dominantes sont "bons" et "moyen+"

gre,mauvais,moyen,moyen +,bon
admit,,,,
0.0,42,71,73,87
1.0,6,31,33,57


In [6]:
niveaux_notes = pd.cut(x = df['gpa'],
                       bins = [2, 2.8, 3.2, 3.6, 4],
                       labels= ['mauvais', 'moyen', 'moyen+', 'bon'])

pd.crosstab(df['admit'], niveaux_notes, normalize='columns') # ou normalize = 1

# Les étudiants ayant un "bon" niveau ont plus de chances d'être admis que la moyenne (0.64 > 0.5).
# Par contre, avoir un niveau "moyen" ou "moyen+" n'est pas très avantageux.

gpa,mauvais,moyen,moyen+,bon
admit,,,,
0.0,0.8,0.757282,0.706667,0.565574
1.0,0.2,0.242718,0.293333,0.434426


In [7]:
nv_rank = pd.cut(x = df['rank'],
                       bins = [0.0, 1.0, 2.0, 3.0, 4.0],
                       labels= ['rk1.0', 'rk2.0', 'rk3.0', 'rk4.0'])

pd.crosstab(df['admit'], nv_rank, normalize='columns')

rank,rk1.0,rk2.0,rk3.0,rk4.0
admit,,,,
0.0,0.459016,0.642384,0.768595,0.820896
1.0,0.540984,0.357616,0.231405,0.179104


In [8]:
df = df.join(pd.get_dummies(niveaux_notes, prefix='niveau'))
df = df.join(pd.get_dummies(test_gre, prefix='gre'))


In [9]:
df

,admit,gre,gpa,rank,niveau_mauvais,niveau_moyen,niveau_moyen+,niveau_bon,gre_mauvais,gre_moyen,gre_moyen +,gre_bon
0,0.0,380.0,3.61,3.0,False,False,False,True,True,False,False,False
1,1.0,660.0,3.67,3.0,False,False,False,True,False,False,False,True
2,1.0,800.0,4.00,1.0,False,False,False,True,False,False,False,True
3,1.0,640.0,3.19,4.0,False,True,False,False,False,False,False,True
4,0.0,520.0,2.93,4.0,False,True,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
395,0.0,620.0,4.00,2.0,False,False,False,True,False,False,True,False
396,0.0,560.0,3.04,3.0,False,True,False,False,False,False,True,False
397,0.0,460.0,2.63,2.0,True,False,False,False,False,True,False,False
398,0.0,700.0,3.65,2.0,False,False,False,True,False,False,False,True


In [10]:
df = df.join(pd.get_dummies(nv_rank, prefix='rk'))

In [11]:
df

,admit,gre,gpa,rank,niveau_mauvais,niveau_moyen,niveau_moyen+,niveau_bon,gre_mauvais,gre_moyen,gre_moyen +,gre_bon,rk_rk1.0,rk_rk2.0,rk_rk3.0,rk_rk4.0
0,0.0,380.0,3.61,3.0,False,False,False,True,True,False,False,False,False,False,True,False
1,1.0,660.0,3.67,3.0,False,False,False,True,False,False,False,True,False,False,True,False
2,1.0,800.0,4.00,1.0,False,False,False,True,False,False,False,True,True,False,False,False
3,1.0,640.0,3.19,4.0,False,True,False,False,False,False,False,True,False,False,False,True
4,0.0,520.0,2.93,4.0,False,True,False,False,False,True,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,0.0,620.0,4.00,2.0,False,False,False,True,False,False,True,False,False,True,False,False
396,0.0,560.0,3.04,3.0,False,True,False,False,False,False,True,False,False,False,True,False
397,0.0,460.0,2.63,2.0,True,False,False,False,False,True,False,False,False,True,False,False
398,0.0,700.0,3.65,2.0,False,False,False,True,False,False,False,True,False,True,False,False


In [12]:
# Séparation des données et des cibles 
data = df.iloc[:,4:13]
target = df['admit']

In [13]:
data.columns


Index(['niveau_mauvais', 'niveau_moyen', 'niveau_moyen+', 'niveau_bon',
       'gre_mauvais', 'gre_moyen', 'gre_moyen +', 'gre_bon', 'rk_rk1.0'],
      dtype='object')

In [14]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=66) 


In [15]:
clf = linear_model.LogisticRegression(C=1.0)
clf.fit(X_train, y_train)


LogisticRegression()

In [16]:

y_pred = clf.predict(X_test)

# Calcul de la matrice de confusion 

## Méthode 1 : à l'aide de sklearn
from sklearn.metrics import confusion_matrix 
cm = confusion_matrix(y_test,y_pred)
print(cm)

[[55  5]
 [13  7]]


In [17]:

## Méthode 2 : à l'aide de pandas
cm = pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite'])
cm

Classe prédite,0.0,1.0
Classe réelle,,
0.0,55,5
1.0,13,7


In [18]:
clf.score(X_test, y_test)

0.775

In [19]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.81      0.92      0.86        60
         1.0       0.58      0.35      0.44        20

    accuracy                           0.78        80
   macro avg       0.70      0.63      0.65        80
weighted avg       0.75      0.78      0.75        80

